In [2]:
import math
from copy import deepcopy
import types

## Helper functions

In [3]:
def is_numeric(x):
    NumberTypes = (types.IntType, types.LongType, types.FloatType)
    return isinstance(x, NumberTypes)

class StrategyExample:
    def __init__(self, func=None):
        if func:
             self.function = func

    def evaluate(self,X):
        return self.function(X)

## Funkcije

In [4]:
def f_test(x):
    if is_numeric(x):
        return (x-4)**2
    elif len(x) == 1:
        return (x[0]-4)**2
    else:
        raise Exception("Argument passed needs to be a list with 1 element or a numeric type")
        
def f1(x):
    if len(x) != 2:
        raise Exception("Argument needs to be a list with two elements")
    return (x[0]-4.)**2 + 4.*(x[1]-2)**2

## Unimodal interval

In [5]:
def find_unimodal(starting_point, function, starting_step=1):
    step = starting_step
    current_point = starting_point
    direction = 1
    if function.evaluate(current_point + step) >= function.evaluate(current_point):
        direction=-1
    next_point = current_point + direction*step
    cnt = 1
    previous_point = current_point
    while function.evaluate(current_point) > function.evaluate(next_point):
        previous_point = current_point
        current_point = next_point
        next_point = starting_point + direction * (2**cnt) * step
        cnt+=1
    
    if previous_point > next_point:
        return next_point, previous_point
    return previous_point, next_point

## Golden section search

In [6]:
def golden_section_search(f,starting_point = None, a=None,b=None,eps=1, verbose = False):
    if starting_point != None:
        a,b = find_unimodal(starting_point,f,eps)
    elif a==None or b== None:
        raise Exception("Starting point or unimodal interval needs to be given")
    
    if verbose:
        print "Searching in interval [%f, %f]" % (a,b)
    
    
    fi = (math.sqrt(5.0) - 1.0)/2
    c = b - (b - a)*fi
    d = a + (b - a)*fi
    while (b - a) > eps:
        if verbose:
            print "|a = %.3f|c = %.3f|d = %.3f|b = %.3f|f(c) = %.3f|f(d) = %.3f|f(c) > f(d) = %s" %(a,c,d,b,f(c),f(d), f(c)>f(d))
        if f.evaluate(c) >= f.evaluate(d):
            a = c
            c = d
            d = a + (b - a)*fi
        else:
            b = d
            d = c
            c = b - (b - a)*fi
    if verbose:
        print ''
        print "Final interval = [",a,", ", b,"]"
    return (a+b)/2.

## Coordinate descend

In [7]:
def coordinate_search(starting_point, epsilon_vector, function):
    if len(starting_point) != len(epsilon_vector):
        raise Exception("Point vector and epsilon vector need to be the same dimension")
        
    coordinate_changed = [False]*len(starting_point)
    min_point = deepcopy(starting_point)
    reset_flag = lambda x: False
    
    cnt = 10
    while any(flag == False for flag in coordinate_changed):
        coordinate_changed = map(reset_flag, coordinate_changed)
        
        def function_1D_wrapper(function,index, array):
            cpy = deepcopy(array)
            def func_1D(x):
                cpy[index] = x
                return function.evaluate(cpy)
            return func_1D
        
        for i,coord in enumerate(min_point):
            func_1D = function_1D_wrapper(function, i, min_point)
            new_coord = golden_section_search(func_1D, coord, eps = epsilon_vector[i])
            min_point[i] = new_coord
            #print new_coord
            if abs(coord-new_coord)<= epsilon_vector[i]:
                coordinate_changed[i] = True
        cnt-=1
        if cnt<0:
            break
        print min_point
    return min_point

# Nelder i Mead simpleks

In [12]:
def nelder_mead(f, x_start,
                step=0.1, no_improve_thr=10e-6,
                no_improv_break=10, max_iter=0,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5):
    
    dim = len(x_start)
    prev_best = f.evaluate(x_start)
    no_improv = 0
    x_start = np.array(x_start)
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = deepcopy(x_start)
        x[i] = x[i] + step
        score = f.evaluate(x)
        res.append([x, score])

    # simplex iter
    iters = 0
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # break after max_iter
        if max_iter and iters >= max_iter:
            return res[0]
        iters += 1

        # break after no_improv_break iterations with no improvement
        print '...best so far:', best

        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0]

        # centroid
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)
        
        # reflection
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr)
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            if escore < res[0][1]:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue
        else:
            if rscore > res[-2][1]:
                if rscore < res[-1][1]:
                    del res[-1]
                    res.append([xr, rscore])
                # contraction
                xc = x0 + rho*(x0 - res[-1][0])
                cscore = f(xc)
                if cscore < res[-1][1]:
                    del res[-1]
                    res.append([xc, cscore])
                    continue
                else:
                    # reduction
                    x1 = res[0][0]
                    nres = []
                    for tup in res:
                        redx = x1 + sigma*(tup[0] - x1)
                        score = f(redx)
                        nres.append([redx, score])
                    res = nres
            else:
                del res[-1]
                res.append([xr, rscore])


if __name__ == "__main__":
    import math
    import numpy as np

    def f(x):
        return math.sin(x[0]) * math.cos(x[1]) * (1. / (abs(x[2]) + 1))

    print nelder_mead(f1, [0., 0.])

...best so far: 30.44
...best so far: 28.5125
...best so far: 26.363125
...best so far: 21.94953125
...best so far: 17.8998828125
...best so far: 12.0610644531
...best so far: 12.0610644531
...best so far: 12.0610644531
...best so far: 9.27703125
...best so far: 9.27703125
...best so far: 7.69966018677
...best so far: 3.41542377472
...best so far: 3.41542377472
...best so far: 0.29453125
...best so far: 0.29453125
...best so far: 0.29453125
...best so far: 0.148687286377
...best so far: 0.10420674324
...best so far: 0.0156961804628
...best so far: 0.0156961804628
...best so far: 0.0156961804628
...best so far: 0.0083095826309
...best so far: 0.00212473189914
...best so far: 0.00212473189914
...best so far: 0.00121428008324
...best so far: 0.000468218209882
...best so far: 0.000468218209882
...best so far: 0.000196898562119
...best so far: 0.000196898562119
...best so far: 0.000117750557909
...best so far: 8.12300926414e-05
...best so far: 5.96450438221e-06
...best so far: 5.96450438221

# Hooke-Jeeves

In [42]:
def hooke_jeeves(starting_point,function,search_direction,decay_coeff=0.95,epsilon=None,max_iter = 0, verbose=False):
    
    iters = 0
    dX = np.array(search_direction).astype(np.float64)
    if epsilon == None:
        epsilon = np.array([1e-6]*len(starting_point))
    else:
        epsilon = np.array(epsilon)
    search_point = deepcopy(np.array(starting_point))
    base_point = deepcopy(np.array(starting_point))
    base_point_score = function(base_point)
    while True:
        if verbose:
            print "Iteration %s base_point = %s" % (iters, base_point)
        if max_iter and iters >= max_iter:
            return base_point
        new_point = istrazi(search_point, function, dX)
        new_point_score = function(new_point)
        if new_point_score < base_point_score:
            search_point = 2 * new_point - base_point
            if (abs(base_point-new_point)<epsilon).all(False):
                return base_point
            base_point = deepcopy(new_point)
            base_point_score = function(base_point)
        else:
            dX *=decay_coeff
            search_point = deepcopy(base_point)
        iters+=1
    pass

def istrazi(point,fun, search_direction):
    x = deepcopy(point)
    dim = len(x)
    for i in range(dim):
        temp = fun(x)
        x[i]+=search_direction[i]
        new = fun(x)
        if new > temp:
            x[i]-=search_direction[i]*2
            new = fun(x)
            if new > temp:
                x[i] += x[i] + search_direction[i]
    return x

hooke_jeeves([0.,0.],f1,[1,1], verbose=True)

Iteration 0 base_point = [ 0.  0.]
Iteration 1 base_point = [ 1.  1.]
Iteration 2 base_point = [ 3.  3.]
Iteration 3 base_point = [ 3.  3.]
Iteration 4 base_point = [ 3.95  2.05]
Iteration 5 base_point = [ 3.95  2.05]
Iteration 6 base_point = [ 3.95  2.05]
Iteration 7 base_point = [ 3.95  2.05]
Iteration 8 base_point = [ 3.95  2.05]
Iteration 9 base_point = [ 3.95  2.05]
Iteration 10 base_point = [ 3.95  2.05]
Iteration 11 base_point = [ 3.95  2.05]
Iteration 12 base_point = [ 3.95  2.05]
Iteration 13 base_point = [ 3.95  2.05]
Iteration 14 base_point = [ 3.95  2.05]
Iteration 15 base_point = [ 3.95  2.05]
Iteration 16 base_point = [ 3.95  2.05]
Iteration 17 base_point = [ 3.95  2.05]
Iteration 18 base_point = [ 3.95  2.05]
Iteration 19 base_point = [ 3.95  2.05]
Iteration 20 base_point = [ 3.95  2.05]
Iteration 21 base_point = [ 3.95  2.05]
Iteration 22 base_point = [ 3.95  2.05]
Iteration 23 base_point = [ 3.95  2.05]
Iteration 24 base_point = [ 3.95  2.05]
Iteration 25 base_point = 

array([ 4.04471684,  1.95528316])

In [61]:
print golden_section_search(f_test,a=2,b=8,eps=1, verbose = True)

Searching in interval [2.000000, 8.000000]
|a = 2.000|c = 4.292|d = 5.708|b = 8.000|f(c) = 0.085|f(d) = 2.918|f(c) > f(d) = False
|a = 2.000|c = 3.416|d = 4.292|b = 5.708|f(c) = 0.341|f(d) = 0.085|f(c) > f(d) = True
|a = 3.416|c = 4.292|d = 4.833|b = 5.708|f(c) = 0.085|f(d) = 0.694|f(c) > f(d) = False
|a = 3.416|c = 3.957|d = 4.292|b = 4.833|f(c) = 0.002|f(d) = 0.085|f(c) > f(d) = False

Final interval = [ 3.416407865 ,  4.2917960675 ]
3.85410196625


In [59]:
print coordinate_search([0.1,0.3],[1e-6,1e-6],f1)

[4.000000219088628, 1.9999999540550042]
[3.999999910071634, 1.9999994540550041]
[3.999999910071634, 1.9999994540550041]


False